In [1]:
import os
import json
import numpy as np
import pandas as pd
import compress_pickle as cpickle

from scripts import utils

In [2]:
model = 'test'
dataset = 'saludmental'

basefolder = f'models/{model}/{dataset}/'

infolder = basefolder + 'input/data/'
configfolder = basefolder + 'config/'

CARGAR DATOS

In [3]:
filename1 = utils.GET_LAST_FILE(infolder, 'dataset')
dataframe = cpickle.load(filename1)

dataframe.columns = dataframe.columns.str.lower()
dataframe = utils.ADD_ROWID(dataframe, 'uniqueid')

columnas_originales = list(dataframe.columns)

DEFINIR TIPO DE DATO

In [4]:
# cargo un diccionario con los tipos de datos
filename2 = utils.GET_LAST_FILE(configfolder + 'sqldtypes_input/', 'table0')
config = utils.READ_JSON(filename2)

# para cada columna, aplico su tipo de dato
for c in list(dataframe.columns):

    dt = config[c]['dtype']

    if dt == 'str':
        dataframe[c] = dataframe[c].astype(str)
    elif dt == 'int':
        dataframe[c] = dataframe[c].astype(int)
    elif dt == 'float':
        dataframe[c] = dataframe[c].astype(float)
    elif dt == 'datetime':
        dataframe[c] = pd.to_datetime(dataframe[c])

FEATURE EXTRACTION

In [5]:
info_columns = ['ruttitular', 'rutbeneficiario', 'numerosolicitud', 
                'idgrupoprestacion','clasificaciongrupo','idsubgrupoprestacion','clasificacionsubgrupo','idaperturaprestacion','clasificacionapertura',
                'fechaprestacion', 'fecharecepcionliquidacion', 'prevision', 'rutprestador', 'nombreprestador']

infoframe = dataframe.loc[:, info_columns]
dataframe = dataframe.drop(info_columns, axis=1)

In [6]:
# codifico columna de prevision
dataframe['fonasa'] = np.where(infoframe['prevision'] == 'FONASA', 1, 0)

In [7]:
# codifico columna de clasificacion prestacion
dataframe = pd.concat([dataframe, pd.get_dummies(infoframe.clasificacionapertura.str.lower(), prefix='clasif', dtype=int)], axis=1)

In [8]:
# codifico columna de dias liquidacion
# dataframe['slaliquidacion'] = (infoframe.fecharecepcionliquidacion - infoframe.fechaprestacion).dt.days / 60

In [9]:
def get_days_dict():

    output = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    
    return output

def get_month_sin(x):

    month = int(x.split('-')[1])
    output = np.sin(((2*np.pi) * (month-1))/11)
    
    return output

def get_month_cos(x):

    month = int(x.split('-')[1])
    output = np.cos(((2*np.pi) * (month-1))/11)

    return output

def get_day_sin(x):
   
    dicto = get_days_dict()

    day = int(x.split('-')[2]) 
    month = int(x.split('-')[1])
    output = np.sin(((2*np.pi) * (day-1))/(dicto[month] - 1))
    
    return output

def get_day_cos(x):
    
    dicto = get_days_dict()

    day = int(x.split('-')[2]) 
    month = int(x.split('-')[1])
    output = np.cos(((2*np.pi) * (day-1))/(dicto[month] - 1))
    
    return output

In [10]:
# codifico el dia de la fecha
dataframe['day_sin'] = infoframe.fechaprestacion.dt.strftime('%Y-%m-%d').apply(lambda x: get_day_sin(x))
dataframe['day_cos'] = infoframe.fechaprestacion.dt.strftime('%Y-%m-%d').apply(lambda x: get_day_cos(x))

# codifico el mes de la fecha
dataframe['month_sin'] = infoframe.fechaprestacion.dt.strftime('%Y-%m-%d').apply(lambda x: get_month_sin(x))
dataframe['month_cos'] = infoframe.fechaprestacion.dt.strftime('%Y-%m-%d').apply(lambda x: get_month_cos(x))

In [11]:
# codifico los ruts
dataframe['titular'] = (infoframe.ruttitular == infoframe.rutbeneficiario).astype(int)

In [12]:
columnas_extraidas = set(dataframe.columns) - set(columnas_originales)

EXPORTAR DATOS PREPARADOS

In [13]:
# almaceno el trabajo de variables
variables = {}
variables['original'] = columnas_originales
variables['information'] = list(info_columns)
variables['extracted'] = list(columnas_extraidas)
variables['selected'] = []

# almaceno la informacion a guardar
dfs = {'dataset': dataframe, 'information': infoframe, 'variables': variables}

In [14]:
# guardo el dataset
filename3 = infolder + 'prepared' + filename1.split('/')[-1].split('.')[0][-14:] + '.lzma'
utils.SAVE_CPICKLE(dfs, filename3)